# Предсказание цены автомобиля c помощью моделей градиентного бустинга

# 1. Подготовка данных

In [2]:
import pandas as pd
import numpy as np
import catboost as cb
import lightgbm as lgb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer

Загрузим данные и просмотрим общую информацию о датасете:

In [3]:
df = pd.read_csv('/datasets/autos.csv', parse_dates=['DateCrawled', 'LastSeen', 'DateCreated']) # прочитаем файл и представим столбцы с датами в формате даты
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31,0,60437,2016-04-06 10:17:21


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null datetime64[ns]
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null datetime64[ns]
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(7), object(6)
memory usage: 43.3+ MB


#### Проверка данных на дубликаты и их удаление

In [5]:
df.duplicated().sum()

4

In [6]:
df = df.drop_duplicates().reset_index(drop=True)

#### Обработка пропусков

In [7]:
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Пропуски присутствуют только в категориальных признаках `VehicleType`, `Gearbox`, `Model`, `FuelType`, `NotRepaired`. Алгоритмы LightGBM и CatBoost умеют работать с пропусками. А вот для использования алгоритма линейной регрессии необходимо заполнить пропуски.  
Создадим копию датафрейма, чтобы не менять данные, на которых обучаются другие алгоритмы.

In [8]:
df1 = df.copy(deep=True)

In [9]:
print('Доля пропусков в VehicleType {:.2%}'.format(df1['VehicleType'].isna().sum()/len(df1)))

Доля пропусков в VehicleType 10.58%


Заполним пропуски в типе кузова `VehicleType` по словарю: на основе бренда и модели выберем самый популярный кузов.  
Создадим словарь с самыми популярными типами кузова (мода) и заполним пропуски этими значениями. Для расчета моды напишем свою функцию, так как нам нужно, чтобы при наличии двух мод в выборке возвращалась только одна из них.

In [10]:
def calc_mode(series):
    """Функция для расчета моды. 
    Принимает на вход столбец датафрейма. 
    Возвращает моду. При наличии двух мод возвращается первая из них.
    """
    mode = series.mode()
    try:
        return mode[0]
    except:
        return mode

mapper = df1.groupby(["Brand","Model"])["VehicleType"].agg(calc_mode).to_dict() # словарь для заполнения пропсуков
 
def map_func(row):
    """Функция заполняет пропуски по словарю. 
    Принимает на вход датафрейм построчно. 
    Возвращает столбец с заполненными пропусками
    """
    if type(row["VehicleType"]) != str:
        return mapper.get((row["Brand"], row["Model"]))
    return row["VehicleType"]

In [12]:
df1["VehicleType"] = df1.apply(map_func, axis=1)

In [13]:
print('Доля оставшихся пропусков в VehicleType {:.2%}'.format(df1['VehicleType'].isna().sum()/len(df1))) # оставшиеся пропуски

Доля оставшихся пропусков в VehicleType 1.93%


Пропуски, которые заполнить не удалось, удалим:

In [14]:
df1 = df1.dropna(subset=['VehicleType']).reset_index(drop=True)

Пропуски в признаках `Model`, `Gearbox` исходя из имеющихся данных корректно заполнить не удастся, удалим эти пропуски:

In [15]:
print('Доля пропусков в Model {:.2%}, в Gearbox {:.2%}'.format(df1['Model'].isna().sum()/len(df1), df1['Gearbox'].isna().sum()/len(df1)))

Доля пропусков в Model 3.71%, в Gearbox 4.84%


In [16]:
df1 = df1.dropna(subset=['Model', 'Gearbox']).reset_index(drop=True)

Пропуски в признаке `FuelType` заполним на "other":

In [17]:
print('Доля пропусков в FuelType {:.2%}'.format(df1['FuelType'].isna().sum()/len(df1)))

Доля пропусков в FuelType 6.19%


In [18]:
df1['FuelType'].value_counts()

petrol      200922
gasoline     92546
lpg           4854
cng            531
hybrid         203
other          108
electric        60
Name: FuelType, dtype: int64

In [19]:
df1['FuelType'] = df1['FuelType'].fillna('other')

Заполним пропуски в признаке `NotRepaired` - допустим, что если клиенты не отмечали наличие ремонтных работ, значит их не было (или они хотят это скрыть:))

In [20]:
print('Доля пропусков в NotRepaired {:.2%}'.format(df1['NotRepaired'].isna().sum()/len(df1)))

Доля пропусков в NotRepaired 16.53%


In [21]:
df1['NotRepaired'] = df1['NotRepaired'].fillna('no')

В целевом признаке `Price` пропусков нет, но есть большое количество нулевых значений цены:

In [22]:
df1[df1['Price'] == 0]['Price'].count()

7156

In [23]:
df1 = df1.loc[df1['Price'] != 0].reset_index(drop=True) # удалим строки с нулевым знач цены

В переменной `RegistrationYear` присутствуют ошибочные значения года, удалим их, оставив промежуток 1950-2020 гг. (с запасом на случай наличия в выборке раритетных авто):

In [24]:
df1['RegistrationYear'].value_counts().sort_index()

1000    1
1400    1
1500    1
1600    1
1602    1
       ..
5911    2
6000    1
6500    1
7100    1
8200    1
Name: RegistrationYear, Length: 103, dtype: int64

In [25]:
df1 = df1.query('RegistrationYear > 1950 & RegistrationYear < 2021')

In [26]:
df1.isna().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

Все пропуски удалены, данные подготовлены к применению алгоритма линейной регрессии.

#### Подготовка признаков. Прямое кодирование (OHE)

Из датасета признаков (X) удалим даты, а также переменные 'PostalCode', 'NumberOfPictures', 'RegistrationMonth' - которые по логическим соображениям не имеют влияния на цену автомобиля.

In [27]:
y = df['Price']
X = df.drop(['Price', 'LastSeen', 'DateCrawled', 'DateCreated', 'PostalCode', 'NumberOfPictures', 'RegistrationMonth'], axis=1)

Закодируем категориальные переменные с помощью прямого кодирования (one-hot encoding) (будем использовать для алгортима LightGBM)

In [28]:
X_ohe = pd.get_dummies(X, dummy_na=True)
X_ohe.head()

,RegistrationYear,Power,Kilometer,VehicleType_bus,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,...,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Brand_nan,NotRepaired_no,NotRepaired_yes,NotRepaired_nan
0,1993,0,150000,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,2011,190,125000,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2004,163,125000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,2001,75,150000,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
4,2008,69,90000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


Разобъем данные на обучающие и тестовые выборки, проверим их размер:

In [29]:
X_train_ohe, X_test_ohe, X_train, X_test, y_train, y_test = \
    train_test_split(X_ohe, X, y, test_size=0.2, random_state=42)

print(X_train_ohe.shape, X_test_ohe.shape, X_train.shape, X_test.shape, y_train.shape[0], y_test.shape[0])

(283492, 318) (70873, 318) (283492, 9) (70873, 9) 283492 70873


Создадим скорер для оценки качества моделей - mse.

In [40]:
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

Сохраним список названий категориальных признаков для CatBoost:

In [31]:
cat_feats = X_train.select_dtypes(include='object').columns.tolist()
cat_feats

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

# 2. Обучение моделей

### LightGBM

1. Используем для обучения данные, закодированные с помощью one-hot encoding

Выполним подбор гиперпараметров с помощью GridSearch:

In [64]:
grid_search = GridSearchCV(estimator = lgb.LGBMRegressor(n_estimators=100), # количество деревьев
                           param_grid = {'num_leaves': [7, 31],# 50], # maximum tree leaves for base learners
                                         'max_depth': [5, 10],
                                         'learning_rate': [0.01, 0.1],# 0.2], # скорость обучения
                                         'subsample': [1]},
                           scoring = mse_scorer,
                           cv = 3, # 3-fold cross validation
                           n_jobs = -1 # количество процессов (-1 means using all processors)
                           )

In [65]:
%%time

grid_search.fit(X_train_ohe, y_train)

CPU times: user 12min 26s, sys: 14.8 s, total: 12min 41s
Wall time: 12min 50s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1], 'max_depth': [5, 10],
                         'num_leaves': [7, 31], 'subsample': [1]},
             pre_dispatch='2

Лучшие подобранные GridSearch гиперпараметры и лучший скорер:

In [67]:
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values('mean_test_score', ascending=False).head()
print(cv_results['params'].iloc[0])
print(np.sqrt(-1*cv_results['mean_test_score'].iloc[0]))

{'learning_rate': 0.1, 'max_depth': 10, 'num_leaves': 31, 'subsample': 1}
1849.5470307129058


Обучим лучшую по итогам GridSearch модель:

In [17]:
estimator_best = lgb.LGBMRegressor(n_estimators=100, **cv_results['params'].iloc[0])

In [18]:
%%time

estimator_best.fit(X_train_ohe, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.2, max_depth=10,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=50, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [19]:
%%time

predictions = estimator_best.predict(X_train_ohe) # предсказание на трейне

CPU times: user 4.41 s, sys: 395 ms, total: 4.81 s
Wall time: 4.81 s


In [20]:
'RMSE на обучающей выборке {} евро'.format(np.sqrt(mean_squared_error(y_train, predictions))) # ошибка на трейне

'RMSE на обучающей выборке 1719.2464355123711 евро'

Проверим качество модели на тестовой выборке:

In [21]:
'RMSE на тестовой выборке {} евро'.format(np.sqrt(mse_scorer(estimator_best, X_test_ohe, y_test)))          

'RMSE на тестовой выборке 1772.0190625144842 евро'

2. Используем оригинальные данные без кодирования категориальных признаков.

Создадим копии датафреймов признаков и заменим тип данных категориальных столбцов на 'category' для использования их в алгоритме LightGBM:

In [22]:
X_train_cat = X_train.copy(deep=True)
X_test_cat = X_test.copy(deep=True)

In [23]:
X_train_cat[cat_feats] = X_train_cat[cat_feats].astype('category')
X_test_cat[cat_feats] = X_test_cat[cat_feats].astype('category')

In [24]:
X_train_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283492 entries, 349735 to 121958
Data columns (total 9 columns):
VehicleType         253528 non-null category
RegistrationYear    283492 non-null int64
Gearbox             267556 non-null category
Power               283492 non-null int64
Model               267764 non-null category
Kilometer           283492 non-null int64
FuelType            257080 non-null category
Brand               283492 non-null category
NotRepaired         226567 non-null category
dtypes: category(6), int64(3)
memory usage: 10.6 MB


Выполним подбор гиперпараметров с помощью GridSearch:

In [25]:
%%time

grid_search.fit(X_train_cat, y_train)

CPU times: user 18min 7s, sys: 5.51 s, total: 18min 13s
Wall time: 18min 21s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [5, 10], 'num_leaves': [7, 31, 50],
                         'subsample': 

Лучшие подобранные GridSearch гиперпараметры:

In [26]:
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values('mean_test_score').head()
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_num_leaves,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
17,21.894741,3.873590,1.572262,0.044843,0.2,10,50,1,"{'learning_rate': 0.2, 'max_depth': 10, 'num_l...",3.032600e+06,3.067935e+06,2.988429e+06,3.029655e+06,32524.978926,18
11,25.949369,3.188743,1.934186,0.042659,0.1,10,50,1,"{'learning_rate': 0.1, 'max_depth': 10, 'num_l...",3.081870e+06,3.124807e+06,3.078706e+06,3.095128e+06,21026.378396,17
16,17.556372,2.374560,1.275738,0.042913,0.2,10,31,1,"{'learning_rate': 0.2, 'max_depth': 10, 'num_l...",3.099138e+06,3.134833e+06,3.089668e+06,3.107880e+06,19446.992316,16
10,19.898285,1.929723,1.594931,0.008768,0.1,10,31,1,"{'learning_rate': 0.1, 'max_depth': 10, 'num_l...",3.172698e+06,3.213591e+06,3.165399e+06,3.183896e+06,21207.952200,15
13,16.656945,2.241913,1.306162,0.002312,0.2,5,31,1,"{'learning_rate': 0.2, 'max_depth': 5, 'num_le...",3.179138e+06,3.211954e+06,3.160647e+06,3.183913e+06,21216.273276,14


Обучим лучшую по итогам GridSearch модель:

In [27]:
estimator_best = lgb.LGBMRegressor(n_estimators=100, **cv_results['params'].iloc[0])

In [28]:
%%time

estimator_best.fit(X_train_cat, y_train)

CPU times: user 37.9 s, sys: 278 ms, total: 38.1 s
Wall time: 38.5 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.2, max_depth=10,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=50, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [29]:
%%time

predictions = estimator_best.predict(X_train_cat) # предсказание на трейне

CPU times: user 4.36 s, sys: 0 ns, total: 4.36 s
Wall time: 4.39 s


In [30]:
'RMSE на обучающей выборке {} евро'.format(np.sqrt(mean_squared_error(y_train, predictions))) # ошибка на трейне

'RMSE на обучающей выборке 1634.7749087272314 евро'

Проверим качество модели на тестовой выборке:

In [31]:
'RMSE на тестовой выборке {} евро'.format(np.sqrt(mse_scorer(estimator_best, X_test_cat, y_test)))          

'RMSE на тестовой выборке 1733.9503735305518 евро'

Качество модели LightGBM лучше при обучении на незакодированных категориальных переменных, когда алгоритм сам обрабатывает категориальные переменные. Лучшая полученная метрика качества на тестовой выборке RMSE = 1734 евро.

### CatBoost

CatBoost работает с категориальными признаками, поэтому обучаем модель на оригинальных признаках без кодирования.  
Для работы CatBoost с пропущенными значениями заполним пропуски новой категорией ("-1"):

In [32]:
X_train = X_train.fillna(-1)
X_test = X_test.fillna(-1)

1. Применим модель CatBoost "из коробки", без подбора параметров, посмотрим на результат:

In [33]:
estimator_cbr = cb.CatBoostRegressor(verbose=0, cat_features=cat_feats)

In [34]:
%%time

estimator_cbr.fit(X_train, y_train) # обучение

CPU times: user 14min 57s, sys: 1min 47s, total: 16min 44s
Wall time: 16min 47s


In [35]:
%%time

predictions = estimator_cbr.predict(X_train) # предсказание на трейне

CPU times: user 3.38 s, sys: 35.9 ms, total: 3.41 s
Wall time: 3.41 s


In [36]:
'RMSE на обучающей выборке {} евро'.format(np.sqrt(mean_squared_error(y_train, predictions))) # ошибка на трейне

'RMSE на обучающей выборке 1786.3599831096665 евро'

Проверим качество модели на тестовой выборке:

In [37]:
'RMSE на тестовой выборке {} евро'.format(np.sqrt(mse_scorer(estimator_cbr, X_test, y_test))) # проверка качества на тестовой выборке

'RMSE на тестовой выборке 1813.1281721959347 евро'

2. Подберем параметры с использованием встроенного в CatBoostRegressor randomized_search:

In [40]:
%%time

estimator_cbr_rs = cb.CatBoostRegressor(iterations=500, cat_features=cat_feats)

grid = {'learning_rate': [0.03, 0.1],
        'depth': [6, 10]}

random_search_result = estimator_cbr_rs.randomized_search(grid, X=X_train, y=y_train)

0:	loss: 1875.3600246	best: 1875.3600246 (0)	total: 6m 59s	remaining: 20m 57s
1:	loss: 1793.1199468	best: 1793.1199468 (1)	total: 13m 56s	remaining: 13m 56s
2:	loss: 1780.0414788	best: 1780.0414788 (2)	total: 28m 10s	remaining: 9m 23s
3:	loss: 1724.3567036	best: 1724.3567036 (3)	total: 43m 9s	remaining: 0us
Estimating final quality...
CPU times: user 1h 37min 59s, sys: 5min 51s, total: 1h 43min 50s
Wall time: 1h 44min 4s


In [41]:
random_search_result['params']

{'depth': 10, 'learning_rate': 0.1}

Обучим лучшую по итогам randomized_search модель:

In [45]:
estimator_cbr_rs = cb.CatBoostRegressor(iterations=500, verbose=100, **random_search_result['params'], cat_features=cat_feats)

In [46]:
%%time

estimator_cbr_rs.fit(X_train, y_train)

0:	learn: 4199.0767308	total: 1.9s	remaining: 15m 45s
100:	learn: 1764.7474249	total: 3m 17s	remaining: 12m 59s
200:	learn: 1678.2411075	total: 6m 42s	remaining: 9m 58s
300:	learn: 1631.0014440	total: 10m 17s	remaining: 6m 48s
400:	learn: 1596.4206192	total: 13m 53s	remaining: 3m 25s
499:	learn: 1565.6682068	total: 17m 27s	remaining: 0us
CPU times: user 16min 29s, sys: 1min 2s, total: 17min 31s
Wall time: 17min 34s


In [48]:
%%time

predictions = estimator_cbr_rs.predict(X_train) # предсказание на трейне

CPU times: user 5.57 s, sys: 0 ns, total: 5.57 s
Wall time: 5.58 s


In [49]:
'RMSE на обучающей выборке {} евро'.format(np.sqrt(mean_squared_error(y_train, predictions))) # ошибка на трейне

'RMSE на обучающей выборке 1587.3672561013127 евро'

Проверим качество модели на тестовой выборке:

In [47]:
'RMSE на тестовой выборке {} евро'.format(np.sqrt(mse_scorer(estimator_cbr_rs, X_test, y_test)))

'RMSE на тестовой выборке 1713.165053208451'

CatBoost неплохо отработала "из коробки", однако с подбором параметров получили результат лучше. Лучшая полученная метрика качества на тестовой выборке RMSE = 1713 евро.

### Линейная регрессия

Обучим модель линейной регрессии. Работаем с подготовленным выше датасетом df1.

In [27]:
y1 = df1['Price']
X1 = df1.drop(['Price', 'LastSeen', 'DateCrawled', 'DateCreated', 'PostalCode', 'NumberOfPictures', 'RegistrationMonth'], axis=1)

Закодируем категориальные переменные (линейная регрессия не умеет работать с категориальными переменными):

In [28]:
X1_ohe = pd.get_dummies(X1)
X1_ohe.head()

,RegistrationYear,Power,Kilometer,VehicleType_bus,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,...,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no,NotRepaired_yes
0,1993,0,150000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
1,2004,163,125000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,2001,75,150000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
3,2008,69,90000,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
4,1995,102,150000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


Разобъем данные на обучающие и тестовые выборки, проверим их размер:

In [38]:
X1_train_ohe, X1_test_ohe, y1_train, y1_test = train_test_split(X1_ohe, y1, test_size=0.2, random_state=42)

print(X1_train_ohe.shape, X1_test_ohe.shape, y1_train.shape[0], y1_test.shape[0])

(249377, 311) (62345, 311) 249377 62345


In [39]:
%%time

estimator_lr = LinearRegression() 
estimator_lr.fit(X1_train_ohe, y1_train) # обучение модели линейной регрессии

CPU times: user 21.5 s, sys: 5.98 s, total: 27.5 s
Wall time: 27.4 s


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [42]:
%%time

predictions = estimator_lr.predict(X1_train_ohe) # предсказание 

CPU times: user 343 ms, sys: 351 ms, total: 694 ms
Wall time: 695 ms


Проверим качество модели на тестовой выборке:

In [43]:
'RMSE на тестовой выборке {} евро'.format(np.sqrt(mse_scorer(estimator_lr, X1_test_ohe, y1_test)))

'RMSE на тестовой выборке 2838.1040556205317 евро'

Линейная регрессия достаточно быстро обучается и предсказывает, но результат у нее очень слабый: RMSE на тесте = 2838 евро, что значительно больше, чем в моделях градиентного бустинга.

# 3. Анализ и сравнение моделей

Лучшее качество мы получили при использовании модели CatBoost с подбором параметров с помощью randomized_search. На тестовой выборке RMSE = 1713 евро. Однако CatBoost достаточно долго обучается (17 минут). Время предсказания 5,6 сек.  

Значительно быстрее обучается (40 сек.) и предсказывает (4,4 сек.) модель LightGBM с подробом параметров с помощью GridSearch. Однако качество мы получили несколько хуже: RMSE = 1734 евро.  

Учитывая важность для клиента качества предсказания, скорости предсказания и времени обучения, для прогноза рыночной стоимости автомобиля рекомендуем использовать модель LightGBM.